In [1]:
from IPython import get_ipython
get_ipython().run_line_magic("reset","-f")

from PyPackageJonas import *

graphics_path = "C:/Users/jonas/Git/Analysis-of-Production-flow-systems-with-focus-on-industry/Latex Thesis/figures/ND examples/Network Gaussian 10"

In [3]:
N = 2
corr = np.array([
    [1,0.99],
    [0.99,1]]
)

In [13]:
Y = np.random.multivariate_normal(np.zeros(2), corr, 10_000).T

In [14]:
# Prepare Kernels, transformed data to uniform (from kernel), Scott's optimal bandwiths
kernels_Y = [gaussian_kde(Y[i,:]) for i in tqdm(range(N))]

U = np.vstack( [np.array([kernels_Y[i].integrate_box_1d(-np.inf, x_i) for x_i in Y[i,:]]) for i in tqdm(range(N)) ])
kernels_U = [gaussian_kde(U[i,:]) for i in tqdm(range(N))]

h = np.array([(kernels_Y[i].covariance**0.5)[0,0] for i in tqdm(range(N))])

100%|██████████| 2/2 [00:00<?, ?it/s]


In [15]:
i,j = 0,1
u_i, u_j = U[i,:], U[j,:]
kernel_ij = gaussian_kde(np.vstack([u_i,u_j]))

# Method 1 - grid approximation

In [26]:
n_mesh = 200
uu1, uu2 = np.meshgrid(
    np.linspace(1/(2*n_mesh),1 - 1/(2*n_mesh),n_mesh),
    np.linspace(1/(2*n_mesh),1 - 1/(2*n_mesh),n_mesh)
)

In [27]:
pdfij = kernel_ij.evaluate(np.vstack([uu1.ravel(), uu2.ravel()]))
pdfij_log = kernel_ij.logpdf(np.vstack([uu1.ravel(), uu2.ravel()]))

In [28]:
vol = 1/n_mesh**2 * pdfij.sum()
MI_1 = 1/n_mesh**2 * (pdfij * pdfij_log).sum() / vol - np.log(vol)

MI_1

# 10:  1.9294946321720645
# 100: 1.8016063783498006
# 200: 1.8016435115350589

1.8016435115350589

# Method 2 - using scipy integrate

In [21]:
print("vol")
d_vol = kernel_ij.integrate_box(
        low_bounds = [0,0],
        high_bounds = [1,1]
    )

print("dblquad")
MI_2 = dblquad(
    lambda x,y : 
        kernel_ij.pdf(np.vstack([x,y])) * \
        kernel_ij.logpdf(np.vstack([x,y])),
    0,1,
    0,1
)[0] / d_vol - np.log(d_vol)

In [22]:
MI_2

1.8016577647107468

In [14]:
import numpy as np
import scipy
(np.array([[1,2,3],[3,4,5]]) * np.linspace(0,1,5)[:,None,None]).sum(axis=0)

array([[ 2.5,  5. ,  7.5],
       [ 7.5, 10. , 12.5]])

In [21]:
scipy.stats.norm().ppf(np.array([0.00001,0.99999]))

array([-4.26489079,  4.26489079])

In [3]:
import numpy as np

In [4]:
M = 100
temp = np.array([[1,M],[0,1]])
temp

array([[  1, 100],
       [  0,   1]])

In [8]:
np.linalg.eig(temp.T @ temp)

EigResult(eigenvalues=array([9.99800050e-05, 1.00019999e+04]), eigenvectors=array([[-0.99995001, -0.0099985 ],
       [ 0.0099985 , -0.99995001]]))

In [6]:
temp @ np.array([0,1])

array([100,   1])